In [ ]:
#Import all libraries
import pandas as pd
import numpy as np
import sys
from itertools import combinations, groupby
from collections import Counter
from IPython.display import display
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
#Input all files
import os
for dirname, _, filenames in os.walk('C:/Users/Wamika/Desktop/Dissertation/Food Data/All csv files'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
df_order = pd.read_csv('C:/Users/Wamika/Desktop/Dissertation/Food Data/All csv files/order_products__train.csv')
df_product = pd.read_csv('C:/Users/Wamika/Desktop/Dissertation/Food Data/All csv files/products.csv')
df_departments = pd.read_csv('C:/Users/Wamika/Desktop/Dissertation/Food Data/All csv files/departments.csv')
df_aisles = pd.read_csv('C:/Users/Wamika/Desktop/Dissertation/Food Data/All csv files/aisles.csv')

prior_data = pd.read_csv('C:/Users/Wamika/Desktop/Dissertation/Food Data/All csv files/order_products__prior.csv')
train_data = pd.read_csv('C:/Users/Wamika/Desktop/Dissertation/Food Data/All csv files/order_products__train.csv')
orders_data = pd.read_csv('C:/Users/Wamika/Desktop/Dissertation/Food Data/All csv files/orders.csv')
products_data = pd.read_csv('C:/Users/Wamika/Desktop/Dissertation/Food Data/All csv files/products.csv')

In [ ]:
orders = pd.read_csv('C:/Users/Wamika/Desktop/Dissertation/Food Data/All csv files/order_products__prior.csv')

In [ ]:
df_orders = pd.merge(df_order, df_product, on='product_id')
df_orders.head(5)

In [ ]:
final_dataframe = pd.merge(pd.merge(pd.merge(df_order, 
                                          df_product, 
                                          on="product_id", how = 'inner'), 
                                   df_aisles, 
                                   on="aisle_id", how='inner'),
                      df_departments, 
                      on="department_id", how = 'inner')

In [ ]:
final_dataframe.head(5)

In [ ]:
df_meta = final_dataframe.groupby('product_id', as_index=False).agg(set)

In [ ]:
df_meta.head(5)

In [ ]:
df_meta['aisle'] = df_meta['aisle'].apply(lambda x: [i.replace(' ','') for i in x])
df_meta['department'] = df_meta['department'].apply(lambda x: [i.replace(' ','') for i in x])

In [ ]:
df_meta['metadata'] = df_meta.apply(lambda x : ' ' + ' '.join(x['aisle']) + ' ' + ' '.join(x['department']), axis = 1)
df_meta.head(5)

In [ ]:
df_vec = df_meta.head(10000) 
df_produtos_previstos = pd.merge(df_vec[['product_id']], df_product[['product_id','product_name']], on='product_id', how='inner')

count_vec = CountVectorizer(stop_words='english')
count_vec_matrix = count_vec.fit_transform(df_vec['metadata'])
cosine_sim_matrix = cosine_similarity(count_vec_matrix, count_vec_matrix)
mapping = pd.Series(df_produtos_previstos.index, index = df_produtos_previstos['product_name'])

In [ ]:
def recommend_products(product_input):
    
    product_index = mapping[product_input]
    
    similarity_score = list(enumerate(cosine_sim_matrix[product_index]))
    similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
    
    similarity_score = similarity_score[1:10]
    product_indices = [i[0] for i in similarity_score]
    
    return (df_vec['product_name'].iloc[product_indices])

In [ ]:
df_produtos_previstos.sample(100)

In [ ]:
recommend_products('Over Tired and Cranky Bubble Bath')

In [ ]:
recommend_products('Chocolate Sandwich Cookies')

In [ ]:
recommend_products('Green Chile Anytime Sauce')